In [2]:
from theano.sandbox import cuda

In [1]:
%matplotlib inline
import utils_modified; reload(utils_modified)
from utils_modified import *
from __future__ import division, print_function

Using gpu device 0: Tesla K80 (CNMeM is disabled)
Using Theano backend.


In [3]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from keras.models import Sequential
from keras.engine.topology import Merge

In [4]:
model_path = '../data/imdb/models/'

In [5]:
from keras.datasets import imdb
idx = imdb.get_word_index()

In [6]:
idx_arr = sorted(idx, key=idx.get)
idx_arr[:10]

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i']

In [7]:
len(idx_arr)

88584

In [8]:
idx2word = {v: k for k, v in idx.iteritems()}

In [9]:
n = 5
(idx2word[n], idx[idx2word[n]])

('to', 5)

In [10]:
path = get_file('imdb_full.pkl',
                origin='https://s3.amazonaws.com/text-datasets/imdb_full.pkl',
                md5_hash='d091312047c43cf9e4e38fef92437263')
f = open(path, 'rb')
(x_train, labels_train), (x_test, labels_test) = pickle.load(f)

In [11]:
# x_train contains lists of variable length were words are represented by indices
len(x_train[0]), len(x_train[1]), len(x_train[2])

(138, 433, 149)

In [12]:
wrds = [idx2word[n] for n in x_train[0]]
' '.join(wrds)

"bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell high's satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers' pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector i'm here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isn't"

In [13]:
vocab_size = 2500
wvect_dims = 32
maxlen = 500

In [14]:
def paddedset(orig_set):
    adj_set = [[min(n,vocab_size-1) for n in sq] for sq in orig_set]
    return pad_sequences(adj_set, maxlen=maxlen, dtype='int32', padding='pre', truncating='post', value=0)

In [15]:
x_train_pad = paddedset(x_train)
x_test_pad = paddedset(x_test)

In [16]:
print(len(x_train[0]))
print(x_train[0][0:8])
print([n for n in x_train_pad[0] if n>0][0:8])

138
[23022, 309, 6, 3, 1069, 209, 9, 2175]
[2499, 309, 6, 3, 1069, 209, 9, 2175]


In [17]:
model1 = Sequential([
        Embedding(vocab_size, wvect_dims, input_length=maxlen),
        Flatten(),
        Dense(100, activation='relu'),
        Dropout(0.7),
        Dense(1, activation='sigmoid')
    ])
model1.compile(optimizer=Adam(), loss='binary_crossentropy') # WARNING : metrics=['accuracy'] fails

In [18]:
demobatchsize = 5
input_array = np.random.randint(vocab_size, size=(demobatchsize, maxlen))
output_array = model1.predict(input_array)
output_array.shape

(5, 1)

In [19]:
model1.fit(x_train_pad, labels_train, batch_size=100, nb_epoch=5, validation_data=(x_test_pad, labels_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 2s - loss: 0.5212 - val_loss: 0.3616
Epoch 2/5
25000/25000 [==============================] - 2s - loss: 0.2618 - val_loss: 0.3030
Epoch 3/5
25000/25000 [==============================] - 2s - loss: 0.1595 - val_loss: 0.3476
Epoch 4/5
25000/25000 [==============================] - 2s - loss: 0.0808 - val_loss: 0.4106
Epoch 5/5
25000/25000 [==============================] - 2s - loss: 0.0362 - val_loss: 0.4903
